In [1]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pickle
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
max_len = 80
okt = Okt()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와',
             '네', '들', '듯', '지', '임', '게', '요', '로', '하고', '으로', '안', '너무', '잘', ]


In [14]:
text = '다른 회사 두유는 비린내??같은게 너무 심해서 못마시는데 매일우유는 고소하고 담백해서 좋아요.'

In [15]:
new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', text)
new_sentence = okt.morphs(new_sentence)
new_sentence = [word for word in new_sentence if not word in stopwords]
encoded = tokenizer.texts_to_sequences([new_sentence])
pad_new = pad_sequences(encoded, maxlen = max_len)

In [17]:
encoded

[[38, 771, 704, 4440, 1066, 1249, 21, 6070, 194, 71, 169, 1361, 8707, 3]]

In [16]:
pad_new

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          38,  771,  704, 4440, 1066, 1249,   21, 6070,  194,   71,  169,
        1361, 8707,    3]], dtype=int32)

In [6]:
# load tokenzier
with open('./GRU_model_200708_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [4]:
new_sentence = ['우리', '카드']
tokenizer.texts_to_sequences([new_sentence])

[[629, 1890]]

In [3]:
from keras.models import load_model
import h5py
import gcsfs
import pickle

PROJECT_NAME = '*********'
CREDENTIALS  = "./***************************.json"
MODEL_PATH   = 'gs://npl_text_v2/GRU_model_200708.h5'
TOKEN_PATH   = 'gs://npl_text_v2/GRU_model_200708_tokenizer.pickle'

FS = gcsfs.GCSFileSystem(project=PROJECT_NAME, token=CREDENTIALS)

In [8]:
with FS.open(TOKEN_PATH, 'rb') as handle:
     tokenizer = pickle.load(handle)

In [9]:
new_sentence = ['우리', '카드']
tokenizer.texts_to_sequences([new_sentence])

[[629, 1890]]

In [12]:
with FS.open(MODEL_PATH, 'rb') as model_file:
    model_gcs = h5py.File(model_file, 'r')
    myModel = load_model(model_gcs)

In [16]:
def sentiment_predict2(new_sentence):
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = max_len)
    score = float(myModel.predict(pad_new))
    if score <= 0.15:
        result = '부정'
    else:
        result = '중립/긍정'
    return result, round( (1-score)*100,2)

In [19]:
max_len = 80
result__ =  sentiment_predict2(new_sentence)

1/1 [==============================] - 0s 41ms/step


In [20]:
result__

('중립/긍정', 64.41)